In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import re

In [2]:
# Load data

df = pd.read_csv('/content/drive/MyDrive/Laptop-Prices/laptop_details.csv')

df.head()

,Product,Rating,MRP,Feature
0,Lenovo IdeaPad 3 Core i3 11th Gen - (8 GB/256 ...,4.2,"₹36,990",Intel Core i3 Processor (11th Gen)8 GB DDR4 RA...
1,Lenovo IdeaPad 3 Core i3 11th Gen - (8 GB/512 ...,4.2,"₹39,990",Intel Core i3 Processor (11th Gen)8 GB DDR4 RA...
2,ASUS VivoBook 15 (2022) Core i3 10th Gen - (8 ...,4.3,"₹32,990",Intel Core i3 Processor (10th Gen)8 GB DDR4 RA...
3,HP Pavilion Ryzen 5 Hexa Core AMD R5-5600H - (...,4.4,"₹49,990",AMD Ryzen 5 Hexa Core Processor8 GB DDR4 RAM64...
4,ASUS TUF Gaming F15 Core i5 10th Gen - (8 GB/5...,4.4,"₹49,990",Intel Core i5 Processor (10th Gen)8 GB DDR4 RA...


In [3]:
# Shape of the dataset

df.shape

(720, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Product  720 non-null    object 
 1   Rating   581 non-null    float64
 2   MRP      720 non-null    object 
 3   Feature  720 non-null    object 
dtypes: float64(1), object(3)
memory usage: 22.6+ KB


In [5]:
# Clean and Convert the MRP feature to an integer

df['MRP'] = df['MRP'].str.replace('₹', '')
df['MRP'] = df['MRP'].str.replace(',', '')

df['MRP'] = df['MRP'].astype('int')
df.MRP.dtype

dtype('int64')

In [6]:
# Check duplicate values
 
df.duplicated().sum()

297

In [7]:
df[df.duplicated()]

,Product,Rating,MRP,Feature
25,RedmiBook Pro Core i5 11th Gen - (8 GB/512 GB ...,4.1,39990,Intel Core i5 Processor (11th Gen)8 GB DDR4 RA...
26,Lenovo IdeaPad 3 Core i3 11th Gen - (8 GB/512 ...,4.2,39990,Intel Core i3 Processor (11th Gen)8 GB DDR4 RA...
27,ASUS ROG Strix G15 (2022) with 90Whr Battery R...,NaN,196990,AMD Ryzen 9 Octa Core Processor16 GB DDR5 RAMW...
29,realme Book (Slim) Core i3 11th Gen - (8 GB/25...,4.4,46990,Stylish & Portable Thin and Light Laptop14 inc...
30,Lenovo IdeaPad 1 Ryzen 5 Quad Core 3500U - (8 ...,4.2,38705,AMD Ryzen 5 Quad Core Processor8 GB DDR4 RAM64...
...,...,...,...,...
708,ASUS VivoBook 15 Core i5 10th Gen - (8 GB/1 TB...,3.9,44990,Intel Core i5 Processor (10th Gen)8 GB DDR4 RA...
710,Lenovo IdeaPad 1 Ryzen 3 Dual Core 3250U - (8 ...,4.2,35590,AMD Ryzen 3 Dual Core Processor8 GB DDR4 RAM64...
711,ASUS ROG Strix G15 Advantage Edition with 90Wh...,4.9,150990,AMD Ryzen 9 Octa Core Processor16 GB DDR4 RAMW...
716,Lenovo IdeaPad Gaming 3 Ryzen 7 Octa Core AMD ...,4.5,73490,AMD Ryzen 7 Octa Core Processor8 GB DDR4 RAM64...


In [8]:
# Drop duplicates

df = df.drop_duplicates(keep='first')     

df.shape

(423, 4)

### **Extracting features using Regex**

In [9]:
print(df.iloc[0,0])
print(df.iloc[0,-1])

print(df.iloc[1,0])
print(df.iloc[1,-1])

Lenovo IdeaPad 3 Core i3 11th Gen - (8 GB/256 GB SSD/Windows 11 Home) 14ITL05 Thin and Light Laptop
Intel Core i3 Processor (11th Gen)8 GB DDR4 RAM64 bit Windows 11 Operating System256 GB SSD35.56 cm (14 inch) DisplayOffice Home and Student 20212 Year Onsite�Warranty
Lenovo IdeaPad 3 Core i3 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) 82H801L7IN | 82H802FJIN | 82H802...
Intel Core i3 Processor (11th Gen)8 GB DDR4 RAM64 bit Windows 11 Operating System512 GB SSD39.62 cm (15.6 inch) DisplayOffice Home and Student 20212 Year Onsite�Warranty


In [10]:
# Extract features

def extract_features(row):
    product = row['Product']
    feature = row['Feature']
    
    ram_size = re.findall(r'(\d+) ?(?:GB|TB)', feature)
    ram_size = int(ram_size[0]) if ram_size else None
    
    ram_type = re.findall(r'(?:LP)?DDR\d\S*|Unified\sMemory', feature)
    ram_type = ram_type[0] if ram_type else None
    
    processor = re.findall(r'(?:AMD|Intel|M\d+|Qualcomm Snapdragon)[\s\w]*\b', feature)
    processor = processor[0] if processor else None

    storage = re.findall(r'(\d+) ?(?:GB|TB) ?(SSD|HDD)', feature)
    storage = list(set([(f"{s[0]} {s[1]}") for s in storage])) if storage else None
    storage = ", ".join(storage) if storage else None
  
    os = re.findall(r'(Windows|Mac OS|Linux|DOS|Chrome)[\s\w]', feature)
    os = os[0] if os else None
    
    brand = re.findall(r'^\w+', product)
    brand = brand[0] if brand else None
    
    return pd.Series([ram_size, ram_type, processor, storage, os, brand],
                     index=['Ram Size', 'Ram Type', 'Processor', 'Storage', 'OS', 'Brand'])
    
df[['RAM_Size(GB)', 'RAM_Type', 'Processor', 'Storage', 'OS', 'Brand']] = df.apply(extract_features, axis=1)

df.sample(3)

,Product,Rating,MRP,Feature,RAM_Size(GB),RAM_Type,Processor,Storage,OS,Brand
224,ASUS Core i5 10th Gen - (8 GB/512 GB SSD/Windo...,4.3,43490,Intel Core i5 Processor (10th Gen)8 GB DDR4 RA...,8,DDR4,Intel Core i5 Processor,512 SSD,Windows,ASUS
159,ASUS TUF Dash F15 Core i7 12th Gen - (16 GB/1 ...,4.5,99990,Intel Core i7 Processor (12th Gen)16 GB DDR5 R...,16,DDR5,Intel Core i7 Processor,1 SSD,Windows,ASUS
662,HP Ryzen 3 Dual Core 3250U - (8 GB/256 GB SSD/...,3.9,31490,AMD Ryzen 3 Dual Core Processor8 GB DDR4 RAM64...,8,DDR4,AMD Ryzen 3 Dual Core Processor8 GB DDR4 RAM64...,256 SSD,Windows,HP


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 0 to 719
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product       423 non-null    object 
 1   Rating        339 non-null    float64
 2   MRP           423 non-null    int64  
 3   Feature       423 non-null    object 
 4   RAM_Size(GB)  423 non-null    int64  
 5   RAM_Type      423 non-null    object 
 6   Processor     423 non-null    object 
 7   Storage       414 non-null    object 
 8   OS            423 non-null    object 
 9   Brand         423 non-null    object 
dtypes: float64(1), int64(2), object(7)
memory usage: 36.4+ KB


In [12]:
df['RAM_Type'].value_counts()

DDR4              320
DDR5               42
LPDDR4X            30
Unified Memory     12
LPDDR5              9
LPDDR4              8
DDR4,               1
LPDDR3              1
Name: RAM_Type, dtype: int64

In [13]:
df['Storage'].value_counts()

512 SSD             223
1 SSD                74
256 SSD              63
1 HDD, 256 SSD       25
1 HDD                14
2 SSD                 6
128 SSD               4
1 HDD, 512 SSD        2
1 HDD, 128 SSD        2
256 HDD, 256 SSD      1
Name: Storage, dtype: int64

In [15]:
df['Brand'].value_counts()

ASUS         136
Lenovo        62
HP            57
DELL          56
acer          34
MSI           29
APPLE         19
Infinix       14
realme         4
ALIENWARE      3
RedmiBook      2
SAMSUNG        2
Ultimus        2
Vaio           1
GIGABYTE       1
Nokia          1
Name: Brand, dtype: int64

In [14]:
df['Processor'].value_counts()

Intel Core i5 Processor                                                                        109
Intel Core i3 Processor                                                                         69
Intel Core i7 Processor                                                                         52
AMD Ryzen 5 Hexa Core Processor8 GB DDR4 RAM64 bit Windows 11 Operating System512 GB SSD39      11
Intel Core i9 Processor                                                                         11
                                                                                              ... 
M1 Max Processor32 GB Unified Memory RAMMac OS Operating System1 TB SSD41                        1
AMD Ryzen 3 Dual Core Processor8 GB DDR4 RAM64 bit Windows 11 Operating System512 GB SSD35       1
AMD Ryzen 5 Quad Core Processor8 GB DDR4 RAM32 bit Windows 11 Operating System512 GB SSD35       1
AMD Ryzen 3                                                                                      1
AMD Ryzen 

In [18]:
# Clean columns

def clean_processor(processor):
    processor = re.sub(r'Processor.*', '', str(processor))
    return processor

df['Processor'] = df['Processor'].apply(clean_processor)
df['Processor'] = df['Processor'].str.rstrip()

df['RAM_Type'] = df['RAM_Type'].str.replace(',', '')

df.head(2)

,Product,Rating,MRP,Feature,RAM_Size(GB),RAM_Type,Processor,Storage,OS,Brand
0,Lenovo IdeaPad 3 Core i3 11th Gen - (8 GB/256 ...,4.2,36990,Intel Core i3 Processor (11th Gen)8 GB DDR4 RA...,8,DDR4,Intel Core i3,256 SSD,Windows,Lenovo
1,Lenovo IdeaPad 3 Core i3 11th Gen - (8 GB/512 ...,4.2,39990,Intel Core i3 Processor (11th Gen)8 GB DDR4 RA...,8,DDR4,Intel Core i3,512 SSD,Windows,Lenovo


In [19]:
df['Processor'].value_counts()

Intel Core i5                   110
Intel Core i3                    70
Intel Core i7                    52
AMD Ryzen 5 Hexa Core            40
AMD Ryzen 7 Octa Core            38
Intel Celeron Dual Core          18
AMD Ryzen 9 Octa Core            16
AMD Ryzen 3 Dual Core            16
Intel Core i9                    11
AMD Ryzen 5 Quad Core             8
M1                                7
M1 Pro                            6
M2                                5
Intel i3                          3
AMD Ryzen 3 Quad Core             3
Intel Pentium Quad Core           2
AMD Athlon Dual Core              2
Intel Celeron Quad Core           2
Intel Pentium Silver              2
AMD Dual Core                     1
Intel i7                          1
AMD Ryzen 5 Dual Core             1
AMD Ryzen 5                       1
AMD Ryzen R5                      1
M1 Max                            1
AMD Ryzen 3                       1
Intel PQC                         1
Qualcomm Snapdragon 7c Gen 2

In [20]:
# Clean 'Processor' further

def clean_processor(processor):
    processor = re.sub(r'Intel i3', 'Intel Core i3', str(processor))
    processor = re.sub(r'Intel i7', 'Intel Core i7', str(processor))
    processor = re.sub(r'Intel Core i7', 'Intel Core i7', str(processor))
    processor = re.sub(r'AMD Dual Core', 'AMD Ryzen 3 Dual Core', str(processor))
    processor = re.sub(r'AMD Ryzen R5', 'AMD Ryzen 5', str(processor))
    return processor

df['Processor'] = df['Processor'].apply(clean_processor)
df['Processor'].value_counts()

Intel Core i5                   110
Intel Core i3                    73
Intel Core i7                    53
AMD Ryzen 5 Hexa Core            40
AMD Ryzen 7 Octa Core            38
Intel Celeron Dual Core          18
AMD Ryzen 3 Dual Core            17
AMD Ryzen 9 Octa Core            16
Intel Core i9                    11
AMD Ryzen 5 Quad Core             8
M1                                7
M1 Pro                            6
M2                                5
AMD Ryzen 3 Quad Core             3
AMD Ryzen 5                       2
Intel Celeron Quad Core           2
AMD Athlon Dual Core              2
Intel Pentium Silver              2
Intel Pentium Quad Core           2
Intel Evo Core i5                 1
Qualcomm Snapdragon 7c Gen 2      1
Intel PQC                         1
AMD Ryzen 3                       1
M1 Max                            1
AMD Ryzen 7 Quad Core             1
AMD Ryzen 5 Dual Core             1
AMD Ryzen 3 Hexa Core             1
Name: Processor, dtype: int6

In [13]:
# Remove unnecessary columns

df = df.drop(['Product','Rating', 'Feature'], axis=1)

print(df.shape)
df.head(5)

(423, 7)


,MRP,RAM_Size(GB),RAM_Type,Processor,Storage(GB),OS,Brand
0,36990,8,DDR4,Intel Core i3,256,Windows,Lenovo
1,39990,8,DDR4,Intel Core i3,512,Windows,Lenovo
2,32990,8,DDR4,Intel Core i3,512,Windows,ASUS
3,49990,8,DDR4,AMD Ryzen 5 Hexa Core,512,Windows,HP
4,49990,8,DDR4,Intel Core i5,512,Windows,ASUS


In [ ]:
df.to_csv('data_etl.csv')